# Partie 2

In [71]:
import pandas as pd
import numpy as np
import statistics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import accuracy_score
from scipy.stats import ttest_1samp 
from scipy.stats import ttest_ind 
from scipy.stats import ttest_rel
from scipy import stats

In [72]:
# Remplacer les caratere spesial par NAN
missing_values = ["?"]
ds = pd.read_csv('salaires.csv', na_values = missing_values)
ds.head()

,age,classe.travail,education,nombre.education,etat.civil,occupation,Lien.parente,race,sexe,heures.par.semaine,pays.natal,salaire
0,90,NaN,HS-grad,9,Veuf,NaN,Pas_dans_famille,Blanc,Feminin,40,United-States,<=50K
1,82,Privee,HS-grad,9,Veuf,Exec-managerial,Pas_dans_famille,Blanc,Feminin,18,United-States,<=50K
2,66,NaN,Some-college,10,Veuf,NaN,Celibataire,Noir,Feminin,40,United-States,<=50K
3,54,Privee,7th-8th,4,Divorce,Machine-op-inspct,Celibataire,Blanc,Feminin,40,United-States,<=50K
4,41,Privee,Some-college,10,Separe,Prof-specialty,Un_enfant,Blanc,Feminin,40,United-States,<=50K


In [73]:
#modifié le type de la colonne salaire, en remplacant le ‘<=50k’ par 0 et ‘>50k’ par 1.
ds['salaire'].replace(['<=50K', '>50K'], [0, 1], inplace=True)
ds

,age,classe.travail,education,nombre.education,etat.civil,occupation,Lien.parente,race,sexe,heures.par.semaine,pays.natal,salaire
0,90,NaN,HS-grad,9,Veuf,NaN,Pas_dans_famille,Blanc,Feminin,40,United-States,0
1,82,Privee,HS-grad,9,Veuf,Exec-managerial,Pas_dans_famille,Blanc,Feminin,18,United-States,0
2,66,NaN,Some-college,10,Veuf,NaN,Celibataire,Noir,Feminin,40,United-States,0
3,54,Privee,7th-8th,4,Divorce,Machine-op-inspct,Celibataire,Blanc,Feminin,40,United-States,0
4,41,Privee,Some-college,10,Separe,Prof-specialty,Un_enfant,Blanc,Feminin,40,United-States,0
...,...,...,...,...,...,...,...,...,...,...,...,...
32556,22,Privee,Some-college,10,Jamais_marie,Protective-serv,Pas_dans_famille,Blanc,Masculin,40,United-States,0
32557,27,Privee,Assoc-acdm,12,Marie_civil,Tech-support,Epouse,Blanc,Feminin,38,United-States,0
32558,40,Privee,HS-grad,9,Marie_civil,Machine-op-inspct,Mari,Blanc,Masculin,40,United-States,1
32559,58,Privee,HS-grad,9,Veuf,Adm-clerical,Celibataire,Blanc,Feminin,40,United-States,0


In [74]:
print('le nombre de lignes dupliquées est: ',ds.duplicated().sum())

le nombre de lignes dupliquées est:  4243


In [75]:
# Quel est la taille du jeu de donn´ees ?

In [76]:
print('la taille du jeu de donn´ees est: ',ds.shape)

la taille du jeu de donn´ees est:  (32561, 12)


In [77]:
# 2. Combien de valeurs manquantes y a-t-il pour chaque colonne ? Quelle est
# la colonne (feature) qui a le plus de valeurs manquantes ?

In [78]:
nan_values= ds.isna().sum()
print('Les valeurs manquantes dans chaque colonne:  \n', nan_values)
print('Les colonnes qui a le plus de valeurs manquantes est: ', nan_values.idxmax())

Les valeurs manquantes dans chaque colonne:  
 age                      0
classe.travail        1836
education                0
nombre.education         0
etat.civil               0
occupation            1843
Lien.parente             0
race                     0
sexe                     0
heures.par.semaine       0
pays.natal             583
salaire                  0
dtype: int64
Les colonnes qui a le plus de valeurs manquantes est:  occupation


In [79]:
# 3. Combien d’hommes y a-t-il dans ce jeu de donn´ees ?

In [80]:
print('Le nombre d homme dans notre jeu de donnees est: ' , ds['sexe'].tolist().count('Masculin'))

Le nombre d homme dans notre jeu de donnees est:  21790


In [81]:
# 4. Quel est l’ˆage le plus ´elev´e et le plus bas de ce jeu de donn´ees ?

In [82]:
print('l’ˆage le plus ´elev´e est: ' , ds['age'].max())
print('l’ˆage le plus bas est: ' , ds['age'].min())

l’ˆage le plus ´elev´e est:  90
l’ˆage le plus bas est:  17


In [83]:
# 5. Combien de femmes travaillent dans le secteur priv´e?

In [84]:
# drop nan values in Two columns by using subset parameter.
#dss=ds.dropna(subset=['sexe','classe.travail'])
#len(ds.loc[ ((ds['sexe']=='Feminin') & (ds['classe.travail']== 'Privee')) ])
print ('Le nombre de femmes travaillent dans le secteur priv´e est: ',
       ds[(ds['sexe'] == "Feminin") & (ds['classe.travail'] == "Privee")]['sexe'].count())

Le nombre de femmes travaillent dans le secteur priv´e est:  7752


In [85]:
# 6. Quelle est la liste des diff´erents pays repr´esent´es dans la colonne repr´esentant
# le pays natal/d’origine de chaque personne de notre jeu de donn´ees?

In [86]:
print(ds['pays.natal'].unique())

['United-States' nan 'Mexico' 'Greece' 'Vietnam' 'China' 'Taiwan' 'India'
 'Philippines' 'Trinadad&Tobago' 'Canada' 'South' 'Holand-Netherlands'
 'Puerto-Rico' 'Poland' 'Iran' 'England' 'Germany' 'Italy' 'Japan' 'Hong'
 'Honduras' 'Cuba' 'Ireland' 'Cambodia' 'Peru' 'Nicaragua'
 'Dominican-Republic' 'Haiti' 'El-Salvador' 'Hungary' 'Columbia'
 'Guatemala' 'Jamaica' 'Ecuador' 'France' 'Yugoslavia' 'Scotland'
 'Portugal' 'Laos' 'Thailand' 'Outlying-US(Guam-USVI-etc)']


In [87]:
#1-Peut-on pr´edire l’ˆage d’une personne (femme) en fonction de son niveau
#d’´education et son nombre d’heures travaill´ees par semaine ? 

In [88]:
Fonction_encodeur=LabelEncoder()
df = pd.read_csv("salaires.csv")
df = df[df['sexe'] == 'Feminin']
ds1 = df[['age', 'education', 'heures.par.semaine']]
ds1['education'] = Fonction_encodeur.fit_transform(ds1['education'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [89]:
X_train, X_test, y_train, y_test = train_test_split(ds1[['education', 'heures.par.semaine']], ds1[['age']], test_size=0.1, random_state=50)

In [90]:
#L'application de la fonction QR:
q, r = np.linalg.qr(X_train)

In [91]:
#Calcule de la fonction Beta:
Bt = np.dot(np.dot(np.linalg.inv(r), q.T), y_train)

In [92]:
prediction = np.dot(X_test, Bt)
print('la différence moyenne au carré entre les valeurs estimées et la valeur réelle. est:',mean_squared_error(prediction, y_test))

la différence moyenne au carré entre les valeurs estimées et la valeur réelle. est: 264.8727197569161


In [93]:
#2- Si le niveau d’´education et le nombre d’heures travaill´ees ´etaient r´eduits `a
#1 seule dimension qui capture les informations importantes des 2 (SVD) ?

In [94]:
svd = TruncatedSVD(1)

In [95]:
X_train2 = svd.fit_transform(X_train)
X_test2 = svd.transform(X_test)

In [96]:
#L'application de la fonction QR:
q2, r2 = np.linalg.qr(X_train2)
Bt2 = np.dot(np.dot(np.linalg.inv(r2), q2.T), y_train)
prediction2 = np.dot(X_test2, Bt2)
print('la différence moyenne au carré entre les valeurs estimées et la valeur réelle. est: ',mean_squared_error(prediction2, y_test))

la différence moyenne au carré entre les valeurs estimées et la valeur réelle. est:  281.6099286620582


In [97]:
#3. Peut-on pr´edire si le salaire d’une femme sera ≤ 50K en fonction de son
#niveau d’´education, son ˆage et son nombre d’heures travaill´ees par semaine
#? V

In [98]:
ds2 = df[['age', 'education', 'heures.par.semaine', 'salaire']]
ds2['education'] = Fonction_encodeur.fit_transform(ds2['education'])
ds2['salaire'].replace(['<=50K', '>50K'],
                        [0, 1], inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [99]:
X_train3, X_test3, y_train3, y_test3 = train_test_split(ds2[['education', 'heures.par.semaine', 'age']],
                                                    ds2[['salaire']], test_size=0.1, random_state=50)

In [100]:
q3, r3 = np.linalg.qr(X_train3)
Bt3 = np.dot(np.dot(np.linalg.inv(r3), q3.T), y_train3)
prediction3 = np.dot(X_test3, Bt3)

In [101]:
prediction3[prediction3 >= 0.5] = 1
prediction3[prediction3 < 0.5] = 0

In [102]:
print('lexactitude obtenu est: ',accuracy_score(prediction3, y_test3))

lexactitude obtenu est:  0.9035250463821892


In [103]:
#1- Quelle est la moyenne des ˆages ? Quelle est la moyenne du nombre d’heures
#travaill´ees par semaine ? et les valeurs m´edianes ? 

In [104]:
print(' la moyenne des ˆages: ', ds['age'].mean(), '\n', 'le mediane d age est: ',ds['age'].median() )
print('la moyenne du nombre d’heures travaill´ees par semaine: ', ds['heures.par.semaine'].mean(), '\n', 'le mediane du nombre d heures est : ',ds['heures.par.semaine'].median() )

 la moyenne des ˆages:  38.58164675532078 
 le mediane d age est:  37.0
la moyenne du nombre d’heures travaill´ees par semaine:  40.437455852092995 
 le mediane du nombre d heures est :  40.0


In [105]:
#2-Quelle sont les ˆages (prendre les 5 premiers) les plus repr´esentatifs de ce
#jeu de donn´ees ? Repr´esentent-ils une tranche d’ˆage en particulier (ex:
#30-35 ans) ?

In [106]:
#ds['age'].value_counts().nlargest(5).sort_values()
list_age= ds['age'].value_counts()[:5].sort_values().index.tolist()
print('les 5 premiers age les plus repr´esentatifsest: ',ds['age'].value_counts()[:5].sort_values().index.tolist())
# create intervals

les 5 premiers age les plus repr´esentatifsest:  [35, 23, 34, 31, 36]


In [107]:
#3- Comment est la variance (`a quel point les donn´ees sont ´eparpill´es/dispers´ees)
#des heures travaill´ees par semaine ? Et les ˆages ?

In [108]:
print('La variance d age: ', statistics.pvariance(ds['age']))
print('la variance de nombre d heures: ', statistics.pvariance(ds['heures.par.semaine']))

La variance d age:  186.05568600783081
la variance de nombre d heures:  152.45431279269025


In [109]:
#4=Est ce que le niveau d’´education, compar´e `a l’age influe le plus sur la valeur
#du salaire ? En d’autres termes est que le niveau d’´education a plus de
#poids sur le salaire que l’age ?

In [110]:
print('La Correlation age_salaire est: ',ds.corr()['salaire']['age'])
print('La Correlation nombre.education_salaire est: ',ds.corr()['salaire']['nombre.education'])

La Correlation age_salaire est:  0.234037102648859
La Correlation nombre.education_salaire est:  0.335153952690943


In [111]:
##############################################
missing_values = ["?"]
dss = pd.read_csv('salaires.csv', na_values = missing_values)
dss['salaire'].replace(['<=50K', '>50K'], [0, 1], inplace=True)
dss[['education']] = dss[['education']].apply(lambda col:pd.Categorical(col).codes)
print('La Correlation age_salaire est: ',dss.corr()['salaire']['age'])
print('La Correlation nombre.education_salaire est: ',dss.corr()['salaire']['education'])

La Correlation age_salaire est:  0.234037102648859
La Correlation nombre.education_salaire est:  0.07931660927729825


In [112]:
#5-Est ce que le niveau d’´education, compar´e au nombre d’heures travaill´e
#par semaine influe le plus sur la valeur du salaire ? En d’autres termes
#est que le niveau d’´education a plus de poids sur le salaire que le nombre
#d’heures travaill´ees par semaine ?

In [113]:
ds.corr()

,age,nombre.education,heures.par.semaine,salaire
age,1.000000,0.036527,0.068756,0.234037
nombre.education,0.036527,1.000000,0.148123,0.335154
heures.par.semaine,0.068756,0.148123,1.000000,0.229689
salaire,0.234037,0.335154,0.229689,1.000000


In [114]:
#6-A quel point l’ˆage est un facteur qui influe le salaire? Pour r´epondre `a `
#cette question vous devez v´erifier que l’ˆage moyen de ceux qui gagne moins
#de 50k est plus bas que ceux qui gagnent plus de 50k.

In [115]:
age_moyen1= ds[(ds['salaire']==0)]['age'].mean()
age_moyen2= ds[(ds['salaire']== 1)]['age'].mean()
print("Lage moyen de ceux qui gagnent <= 50K: ",age_moyen1)
print("Lage moyen de ceux qui gagnent > 50K: ",age_moyen2)
if age_moyen1 < age_moyen2:
    print('Effectivement, l’ˆage est un facteur qui influe le salaire')
else:
    print('l’ˆage n`est pas vraiment un facteur qui influe le salaire')

Lage moyen de ceux qui gagnent <= 50K:  36.78373786407767
Lage moyen de ceux qui gagnent > 50K:  44.24984058155847
Effectivement, l’ˆage est un facteur qui influe le salaire


In [116]:
#7-A quel point le nombre d’heures travaill´e par semaine est un facteur qui `
#influe le salaire? Pour r´epondre `a cette question vous devez v´erifier que le
#nombre moyen d’heure travaill´e par semaine pour ceux qui gagnent moins
#de 50k est plus bas que ceux qui gagnent plus de 50k.

In [117]:
heure_moyen1= ds[(ds['salaire']==1)]['heures.par.semaine'].mean()
heure_moyen2= ds[(ds['salaire']==0)]['heures.par.semaine'].mean()
print("nombre moyen d’heure travaill´e par semaine pour ceux qui gagnent <= 50K: ",heure_moyen1)
print("nombre moyen d’heure travaill´e par semaine pour ceux qui gagnent > 50K: ",heure_moyen2)
if heure_moyen1 < heure_moyen2:
    print('Effectivement, le nombre d’heures travaill´e par semaine est un facteur qui influe le salaire')
else:
    print('le nombre d’heures travaill´e par semaine ne est pas un facteur qui influe le salaire')

nombre moyen d’heure travaill´e par semaine pour ceux qui gagnent <= 50K:  45.473026399693914
nombre moyen d’heure travaill´e par semaine pour ceux qui gagnent > 50K:  38.840210355987054
le nombre d’heures travaill´e par semaine ne est pas un facteur qui influe le salaire


In [118]:
#8-Est ce qu’il y a une diff´erence de salaire entre les personnes mari´ees et les
#personnes c´elibataires ? Quel groupe gagne le mieux leurs vies ?

In [119]:
celibataires_list = ['Veuf' ,'Divorce', 'Separe' ,'Jamais_marie', 'Conjoit_fait']
celibataires_salaire= ds[ds['etat.civil'].isin(celibataires_list)]['salaire'].value_counts()[1]
print('le nombre de celibataires qui gagne plus de 50kest: ',celibataires_salaire)

ds_marie=ds.loc[(ds['etat.civil'] == 'Marie_civil')| (ds['etat.civil'] == 'Marie_epoux_absent')]
marie_salaire=ds_marie['salaire'].value_counts()[1]
print('le nombre de maries qui gagne plus de 50k est: ',marie_salaire)

print('la moyen de celibataires qui gagne plus de 50kest: ', (celibataires_salaire)/(ds['etat.civil'].isin(celibataires_list).count()))
print('la moyen nombre de maries qui gagne plus de 50k est:', (marie_salaire)/ds_marie['etat.civil'].count())

if (marie_salaire)/ds_marie['etat.civil'].count() > (celibataires_salaire)/(ds['etat.civil'].isin(celibataires_list).count()):
    print('Le groupe qui gagne plus est: les personnes mari´ees')
else:
    print('Le groupe qui gagne plus est: personnes c´elibataires')


le nombre de celibataires qui gagne plus de 50kest:  1115
le nombre de maries qui gagne plus de 50k est:  6726
la moyen de celibataires qui gagne plus de 50kest:  0.03424342004238199
la moyen nombre de maries qui gagne plus de 50k est: 0.43692347667922565
Le groupe qui gagne plus est: les personnes mari´ees


In [120]:
#9-Quelle est la proportion des hommes qui ont un doctorat et gagnent plus
#de 50k ? et chez les femmes ?

In [121]:
print('le nombre des hommes qui ont un doctorat et gagnent plus de 50k est: ',
       ds[(ds["sexe"]=="Masculin")  & (ds["education"] =="Doctorate")& (ds["salaire"] == 1)].shape[0])
print('le nombre des femmes qui ont un doctorat et gagnent plus de 50k est: ',
        ds[(ds["sexe"]=="Feminin")  & (ds["education"] =="Doctorate")& (ds["salaire"] == 1)].shape[0])

print('la proportion des hommes qui ont un doctorat et gagnent plus de 50k est: ',
        (256 / ds[(ds["sexe"]=="Masculin")].shape[0]) * 100)
print('la proportion des femme qui ont un doctorat et gagnent plus de 50k est: ',
        (50 / ds[(ds["sexe"]=="Feminin")].shape[0]) * 100)

le nombre des hommes qui ont un doctorat et gagnent plus de 50k est:  256
le nombre des femmes qui ont un doctorat et gagnent plus de 50k est:  50
la proportion des hommes qui ont un doctorat et gagnent plus de 50k est:  1.1748508490133087
la proportion des femme qui ont un doctorat et gagnent plus de 50k est:  0.46420945130442853


In [122]:
#10. Quelle est la proportion des femmes mari´ees qui n’ont pas fait de masters
#ni de doctorat ? et chez les hommes ?

In [123]:
print('le nombre des hommes mari´ees qui n’ont pas fait de masters ni de doctorat est: ',
      ds[(ds['sexe']=='Masculin')& (ds["etat.civil"] == "Marie_civil") &(ds['education']!='Doctorate') & (ds['education']!='Masters')].shape[0])
print("le nombre des femmes mari´ees qui n’ont pas fait de masters ni de doctorat est: ",
      ds[(ds['sexe']=='Feminin')& (ds["etat.civil"] == "Marie_civil") &(ds['education']!='Doctorate') &(ds['education']!='Masters')].shape[0])
print('la proportion des hommes qui ont un doctorat et gagnent plus de 50k est: ',
      (ds[(ds['sexe']=='Masculin')& (ds["etat.civil"] == "Marie_civil") &(ds['education']!='Doctorate') & (ds['education']!='Masters')].shape[0] /  ds[(ds['sexe']=='Masculin')].shape[0]) * 100)
print('la proportion des femmes qui ont un doctorat et gagnent plus de 50k est: ',
        ( ds[(ds['sexe']=='Feminin')& (ds["etat.civil"] == "Marie_civil") &(ds['education']!='Doctorate') &(ds['education']!='Masters')].shape[0] /  ds[(ds['sexe']=='Feminin')].shape[0]) * 100)

le nombre des hommes mari´ees qui n’ont pas fait de masters ni de doctorat est:  12163
le nombre des femmes mari´ees qui n’ont pas fait de masters ni de doctorat est:  1524
la proportion des hommes qui ont un doctorat et gagnent plus de 50k est:  55.819183111519045
la proportion des femmes qui ont un doctorat et gagnent plus de 50k est:  14.149104075758984


In [124]:
#11. Les ressortissants de quel pays ont les salaires les plus ´elev´es uniquement
#dans notre ´echantillon?

In [125]:
ds_ressortissants=ds.groupby('pays.natal')['salaire'].mean()
print('Les ressortissants de ', ds_ressortissants.idxmax(), ' ont les salaires les plus ´elev´es')

Les ressortissants de  Iran  ont les salaires les plus ´elev´es


In [126]:
#12. Quelle race a les salaires les plus ´elev´es uniquement dans notre ´echantillon?

In [127]:
ds_race=ds.groupby('race')['salaire'].mean()
print('La race ', ds_race.idxmax(), ' ont les salaires les plus ´elev´es')

La race  Asiatique  ont les salaires les plus ´elev´es


# 6.5 Tests d'hypothèse :

In [128]:
#1. Est-ce qu’on peut dire que la moyenne d’heures travaill´ees par semaine est
#autour de 40h/semaine ? Est-ce que l’affirmer serait significatif ?

In [129]:
tset, pval = ttest_1samp(ds['heures.par.semaine'],40)
print("p-values",pval)
if pval < 0.05 :
  print("rejetter l'hypothèse nulle")
else :
  print("on accepte l'hypothèse nulle")

p-values 1.6483286793366825e-10
rejetter l'hypothèse nulle


In [130]:
#2) Est ce que les hommes ont en moyenne un salaire supérieur à 50k comparé
#aux femmes ?

In [131]:
from scipy.stats import ttest_ind 

In [132]:
Fonction_encodeur=LabelEncoder()

In [134]:
nv_df= pd.read_csv("salaires.csv")

In [135]:
nv_df['salaire'] = Fonction_encodeur.fit_transform(nv_df['salaire']== ">50K")
data_female = nv_df[(nv_df['sexe'] == 'Feminin')]
data_m=nv_df[(nv_df['sexe'] == 'Masculin')]


In [136]:
ttest2, pval2 = ttest_ind(data_m["salaire"], data_female["salaire"])
print("p-values",pval2)

p-values 0.0


In [137]:
if pval2 < 0.05 :
  print("rejetter l'hpothèse nulle")
else:
  print("on accepte l'hypothèse nulle")


rejetter l'hpothèse nulle


In [138]:
#3) Est ce qu’il y a un lien quelconque entre l’age et le nombre d’heures travaillé par semaine ?

In [139]:
data_h=ds['heures.par.semaine']
data_a=ds['age']

In [140]:
ttest3, pval3 = stats.ttest_rel(data_h, data_a)
print("p-values",pval3)

if pval3 < 0.05 :
  print("rejetter l'hpothèse nulle")
else:
  print("on accepte l'hypothèse nulle")

p-values 6.751525347329476e-79
rejetter l'hpothèse nulle
